# MP 1: Safety Analysis of Autonomous Vehicles
## Part 3: Combine Analysis of Simulation Data and Real Data
Group 9: 
- Zach Larson (zlarson2)
- Máximo Rojas (mgr9)
- Matthew Paul (mjpaul3)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import scipy

In [3]:
av_data = pd.read_csv("Data Part 3/mp1_part3_data_2024.csv")
print(av_data.shape)
av_data.head()

(2000, 4)


,Unnamed: 0,weather,scenario,status
0,0,clear,ghost_cutin,Completed
1,1,cloudy,ghost_cutin,Completed
2,2,rain,ghost_cutin,Completed
3,3,snow,ghost_cutin,Completed
4,4,clear,ghost_cutin,Completed


**1. Parse the provided Carla simulation dataset and calculate the following probabilities for
the cut-in scenario (you need to filter our invalid data points before doing the analysis):**

In [6]:
filtered_av_data = av_data[av_data['status']!='Failed -- Simulator crashed']
print(filtered_av_data.shape)
filtered_av_data.head()

(1980, 4)


,Unnamed: 0,weather,scenario,status
0,0,clear,ghost_cutin,Completed
1,1,cloudy,ghost_cutin,Completed
2,2,rain,ghost_cutin,Completed
3,3,snow,ghost_cutin,Completed
4,4,clear,ghost_cutin,Completed


>**a. The probability of accident P(acc=1) across all weather conditions. (1 point)**

In [24]:
p_accident = filtered_av_data[filtered_av_data['status']=='Failed -- Accident'].shape[0] / filtered_av_data.shape[0]
print("P(acc = 1):",p_accident)

P(acc=1): 0.0898989898989899


>**b. The probability of an accident conditioned on the weather,
P(acc=1 | weather=?), for each weather condition. (1 point)**

In [10]:
clear_dataframe = filtered_av_data[filtered_av_data['weather']=='clear']
cloudy_dataframe = filtered_av_data[filtered_av_data['weather']=='cloudy']
rain_dataframe = filtered_av_data[filtered_av_data['weather']=='rain']
snow_dataframe = filtered_av_data[filtered_av_data['weather']=='snow']

In [11]:
p_accident_given_clear = clear_dataframe[clear_dataframe['status']=='Failed -- Accident'].shape[0] / clear_dataframe.shape[0]
p_accident_given_cloudy = cloudy_dataframe[cloudy_dataframe['status']=='Failed -- Accident'].shape[0] / cloudy_dataframe.shape[0]
p_accident_given_rain = rain_dataframe[rain_dataframe['status']=='Failed -- Accident'].shape[0] / rain_dataframe.shape[0]
p_accident_given_snow = snow_dataframe[snow_dataframe['status']=='Failed -- Accident'].shape[0] / snow_dataframe.shape[0]

In [25]:
print("P(acc = 1 | weather = clear)",p_accident_given_clear)
print("P(acc = 1 | weather = cloudy)",p_accident_given_cloudy)
print("P(acc = 1 | weather = rain)",p_accident_given_rain)
print("P(acc = 1 | weather = snow)",p_accident_given_snow)

P(acc = 1 | weather = clear) 0.008130081300813009
P(acc = 1 | weather = cloudy) 0.04251012145748988
P(acc = 1 | weather = rain) 0.12474849094567404
P(acc = 1 | weather = snow) 0.18309859154929578


**2. The baseline simulated dataset contains accident information for snowy conditions and
rainy conditions. In California it is sunny 80% of the time, rainy 5% of the time, snowy 2%
of the time and cloudy the rest of the time. In Chicago, it is sunny 60% of the time, it rains
15% and it is snowy 20% of the time, and cloudy the rest of the time. Calculate the
probability of an accident in the cut-in scenario for California and Chicago, respectively.
Clearly state your assumptions and method. (2 points)**

In [14]:
p_clear_cali = .8
p_cloudy_cali = .13
p_rain_cali = .05
p_snow_cali = .02

p_clear_chi = .6
p_cloudy_chi = .05
p_rain_chi = .15
p_snow_chi = .2

Assumptions:
>

Method:

>Law of Total Probability: P(acc) = P(clear)*P(acc|clear) + P(cloudy)*P(acc|cloudy) + P(rain)*P(acc|rain) + P(snow)*P(acc|snow)

In [15]:
p_accident_cali = (p_clear_cali * p_accident_given_clear) + (p_cloudy_cali * p_accident_given_cloudy) + (p_rain_cali * p_accident_given_rain) + (p_snow_cali * p_accident_given_snow)
p_accident_chi = (p_clear_chi * p_accident_given_clear) + (p_cloudy_chi * p_accident_given_cloudy) + (p_rain_chi * p_accident_given_rain) + (p_snow_chi * p_accident_given_snow)

In [26]:
print("P(acc = 1) in California:",p_accident_cali)
print("P(acc = 1) in Chicago:",p_accident_chi)

P(acc = 1) in California: 0.02192977720839371
P(acc = 1) in Chicago: 0.062335546805072556


**3. In Part 2, Task 3.4, you calculated the AV’s probability of an accident per mile for the
California DMV dataset. Suppose you want to compare the simulated accident rate with
the real dataset accident rate. (6 points)**

**Hint: For this question, you can assume that the probability you calculated in Part
2, Task 3.4 is the AV’s marginal (unconditional) accident rate per mile for the real
CA DMV dataset. You can also assume that there is one cut-in scenario (run) per
one mile in the simulation data.**

>**a. Unfortunately, the real DMV only has sunny and cloudy weather:**
>>**i. How would you make a reasonable comparison between the probability of
an accident of the simulated dataset and the real dataset in this case? (1
point)**

remove rain and snow data points from simulated dataset

>>**ii. Compare the two probabilities and report your findings. (1 point)**

In [21]:
# p_slow_react = (P_rt_greater_given_clear*P_clear) + (P_rt_greater_given_cloudy*P_cloudy)
# p_automatic_disengagement_per_mile = 0.0059

# p_accident = p_automatic_disengagement_per_mile * p_slow_react
# print("Probability of an accident: ",p_accident)

# Probability of an accident:  0.0033509127789046653

# TODO: remove rain and snow points from above and calculate probability

p_clear_cali_norm = .8 / (p_clear_cali + p_cloudy_cali)
p_cloudy_cali_norm = .13 / (p_clear_cali + p_cloudy_cali)
p_accident_cali_clear_cloudy = (p_clear_cali_norm * p_accident_given_clear) + (p_cloudy_cali_norm * p_accident_given_cloudy)
print(p_accident_cali_clear_cloudy)

0.01293589336572483


sim is much higher

>**b. Suppose that there are k cut-in scenarios per one mile, and the probability of having
an accident in one cut-in scenario is p. What is the probability to have at least one
accident per one mile? State your reasoning in detail. (2 points)**

Binomial
1 - p^0(1-p)^k

>**c. Why do you think the probabilities of accident are so different between the
simulation and the rea-world data? (1 point)**

real world probabilty is calculated with an arbitrary reaction time
rw: ASSUMING: disengagement + slow reaction ==> accident (always)
rw: chance of disengagement .0020, 

the prior weather probabilities in the simulation probably do not match up at all to the real world
simulation had equal distibution of weather conditions



d. What are the pros and cons of the simulation data given the discrepancy
between probability of accident in simulation and the real-world data? (1 point)

pros:

simulation accounds for more varied weather conditions
easier to generate simulation data
can control other variables when collecting data


cons:

not directly translatable to real world scenario
inacurate due to model assumptions that are not present in real world

**4. What are your suggestions to improve MP1? What difficulties did you encounter? Please
make sure each member individually (mentioning the member's name) lists their
suggestions for improvements on that problem.**

Matthew Paul

>Be more clear in part 1 which parts of the dataset to use in each question.

Zach Larson

>Give recomended resources and libraries.

Máximo Rojas

>Give template at start of MP instead of right before the deadline.
